In [102]:
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./train_E6oV3lV.csv')
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [103]:
pos_hashtags = []
for tweet in df[df['label']==0]['tweet']:
    regex_list = re.findall("#(\w+)",tweet)
    pos_hashtags = pos_hashtags + regex_list

In [104]:
neg_hashtags = []
for tweet in df[df['label']==1]['tweet']:
    regex_list = re.findall("#(\w+)",tweet)
    neg_hashtags = neg_hashtags + regex_list

In [105]:
Total_hashtags = pos_hashtags + neg_hashtags

### Creating a frequency distribution of Hashtags
Instead of TF-IDF this could be used as coefficients for hashtags

In [106]:
import nltk
pos_hash_dict = nltk.FreqDist(pos_hashtags)
neg_hash_dict = nltk.FreqDist(neg_hashtags)

for x in pos_hash_dict:
    pos_hash_dict[x] = pos_hash_dict[x]/float(len(pos_hashtags))
for x in neg_hash_dict:
    neg_hash_dict[x] = neg_hash_dict[x]/float(len(neg_hashtags))

## Building a word2vec model using gensim

In [107]:
import gensim
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
LabeledSentence = gensim.models.doc2vec.LabeledSentence

def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized;

hashtags_train = labelizeTweets(Total_hashtags, 'TRAIN')

model = Word2Vec(size=100 , min_count=10)
model.build_vocab([[x] for x in Total_hashtags])
model.train([[x] for x in Total_hashtags], total_examples=model.corpus_count, epochs=model.epochs)


0it [00:00, ?it/s]
17813it [00:00, 176789.38it/s]
36853it [00:00, 180289.03it/s]
54901it [00:00, 180300.07it/s]
66879it [00:00, 116575.66it/s]
76296it [00:00, 141577.81it/s]

(161615, 381480)

In [108]:
model.wv.most_similar(positive='enjoy')

[('musicvideo', 0.3074302077293396),
 ('instadaily', 0.3038012385368347),
 ('flagday', 0.2752704620361328),
 ('namaste', 0.2631492614746094),
 ('youtuber', 0.2532731294631958),
 ('enteainment', 0.2512967586517334),
 ('xoxo', 0.24863505363464355),
 ('tears', 0.24853824079036713),
 ('boy', 0.23527175188064575),
 ('podcast', 0.23180250823497772)]

In [110]:
model['enjoy']

array([-3.4706900e-03,  2.0637142e-03, -1.9378130e-03, -4.0441374e-03,
        1.8650873e-03,  1.5402967e-03, -1.5719584e-03, -4.2860387e-03,
       -6.2611463e-05, -2.2159289e-03,  4.8616836e-03,  4.7988533e-03,
       -3.8754351e-03,  1.4601762e-03, -1.9828053e-03,  1.2110966e-04,
       -4.4379854e-03,  4.5729633e-03, -1.4927547e-03,  4.7749178e-03,
        3.8615107e-03,  2.1454906e-03, -1.0454686e-03,  3.8459681e-03,
       -2.9177987e-03, -2.1215116e-03,  1.8458575e-03, -3.1540061e-03,
       -2.8406165e-03, -3.5182209e-04, -2.6022394e-03, -4.7477777e-03,
        4.0419381e-03, -1.3249054e-03, -3.2001098e-03,  2.7428924e-03,
       -4.0643991e-04, -5.5177539e-04,  2.3338906e-03,  4.4600298e-03,
        1.7404994e-03,  2.2918819e-03,  3.2997413e-03,  1.8202115e-03,
        4.2250315e-03, -4.8950124e-03,  4.9957205e-03, -8.8936424e-05,
        4.9798172e-03,  2.2057013e-03,  1.3037844e-03, -1.7273304e-03,
        3.6400722e-03, -4.9177213e-03, -3.1147045e-04,  2.5854004e-03,
      